# Escuela Politécnica Nacional

### Objetivos

Este proyecto tiene como objetivo medir el tiempo promedio que los estudiantes de la EPN que ingresarone el semestre 2012-A (pertenecientes a la primera promoción del SENECYT)se tardaron en culminar su carrera y encontrar si existe una relación entre la carrera y el tiempo de graduación.
Se debe tener en cuenta que como se trabaja con la lista de planes de tesis, los estudiantes considerados en el anális son solo los que se graduan mediante el proceso de titulación por proyecto.

Librerías utilizadas

In [48]:
import tabula
import pandas as pd

### Parámetros y configuraciones

In [49]:
FILE_PATH_STUDENT = "inputs/APROBADOS INGENIERIA_2012.pdf"
FILE_PATH_STUDENT_HTML = "inputs/planesTesis.html"
FILE_PATH_STUDENTS_CSV = "epnStudents2021.csv"

## Extracción

Para realizar este análisis, se toma información de dos fuentes: 
- Lista de estudiantes aprobados para inicar el prepólitecnico en el semestre 2012 - A en formato PDF 
-  Lista de todos los planes aprovados disponible en la página del saew [saew](https://saew.epn.edu.ec/SAETESIS/BusquedaPlanTesis.aspx) en formato HTML

### Extración de información de estudiantes aprovados para el 2012-A en formato PDF

Leer información del pdf y convertir en archivo CSV

In [50]:
def getInfoFromPdf():
    tabula.convert_into(FILE_PATH_STUDENT, 'epnStudents2021.csv', pages='all')

Método para generar el dataset a partir un CSV

In [51]:
def getDatasetFromCSVFile(file_path_param):
    data_set = pd.read_csv(FILE_PATH_STUDENTS_CSV)
    return data_set

Método para generar el correo electrónico de los estudiantes, este campo será utilizado para hacer match
con el otro dataset

In [52]:
def addEmail(df):
    names = df['NOMBRES']
    apellidos = df['APELLIDOS']

    # Generar los correos electrónicos
    correos = []
    for nombre, apellido in zip(names, apellidos):
        nombre_partes = nombre.lower().split()
        apellido_partes = apellido.lower().split()

        # Construir el correo electrónico
        correo = f"{nombre_partes[0]}.{apellido_partes[0]}@epn.edu.ec"
        correos.append(correo)

    df['Email'] = correos
    return df

Método que retorna un data set con la información <br>
[Nombre , Carrera, Email]

In [53]:
def getInfo2012Students():
    getInfoFromPdf()
    info_students_from_pdf = getDatasetFromCSVFile(FILE_PATH_STUDENTS_CSV)
    info_studens_with_email = addEmail(info_students_from_pdf)
    return info_studens_with_email
    

In [54]:
students_2012_info = getInfo2012Students()

In [55]:
print(students_2012_info)

            APELLIDOS               NOMBRES                    CARRERA  \
0       ABARCA ROBLES           JUAN DANIEL  INGENIERIA AGROINDUSTRIAL   
1     ACOSTA CEVALLOS        STALIN ROBERTO                 MATEMATICA   
2      ACOSTA ORDOÑEZ  CHRISTOPHER LEONARDO        INGENIERIA MECANICA   
3       ACOSTA ROSERO           LUIS FELIPE        INGENIERIA MECANICA   
4       ACUÑA MONTERO         MARLON ANDRES                     FISICA   
..                ...                   ...                        ...   
768    ZAPATA HERRERA      CHRISTIAN JAVIER        INGENIERIA MECANICA   
769    ZAPATA HIDALGO        JOSSELYN PAOLA  INGENIERIA AGROINDUSTRIAL   
770      ZAPATA TERAN     STEPHANY CAROLINA     INGENIERIA EMPRESARIAL   
771  ZURITA FERNANDEZ         DIEGO ESTEBAN    INGENIERIA EN PETROLEOS   
772       ZURITA YUGS          KEVIN JAVIER        INGENIERIA MECANICA   

                             Email  
0           juan.abarca@epn.edu.ec  
1         stalin.acosta@epn.edu.ec  


### Extración de información los planes de proyecto de titulación en formato HTML

Método para extraer información del archivo html, donde se encuentran los planes de tesis


In [56]:
def getInfoTesisFromHTML():
    dataframes = pd.read_html(FILE_PATH_STUDENT_HTML)

    columnEmail = dataframes[0]
    columnsName = ['Carrera','FecGradoOral','Email']
    new_emp_list = []

    for i in range(len(columnEmail)):
        carrera = columnEmail.iloc[i]['Carrera']
        fecGradoOral = columnEmail.iloc[i]['FecGradoOral']
        correos = columnEmail.iloc[i]['Email']
        for correo in correos.split():
            if "@epn.edu.ec" in correo:
                #print(correo)
                new_emp_list.append([carrera, fecGradoOral, correo])
    return pd.DataFrame(new_emp_list, columns=columnsName)

In [57]:
students_and_tesis = getInfoTesisFromHTML()
print(students_and_tesis)

                                                Carrera        FecGradoOral  \
0                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
1                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
2     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
3     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
4     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
...                                                 ...                 ...   
8385             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL  17/08/2018 0:00:00   
8386             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
8387             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
8388      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   
8389      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   

                           Email  
0     juan.zambr

## Transformación

Para el proceso de transformación se procede a intersecar los dos dataframes, mediante el correo institucional

Método que genera un nuevo dataFrame realizando una intersección entre los dos dataFrames en base al campo email

In [58]:
def generar_dataframe_interseccion(df1, df2):
    # Obtiene los correos electrónicos únicos en ambos DataFrames
    emails_df1 = set(df1['Email'])
    emails_df2 = set(df2['Email'])
    
    
    # Encuentra los correos electrónicos que se repiten en ambos DataFrames
    emails_repetidos = list(emails_df1.intersection(emails_df2))
    print(emails_repetidos)
    
    # Filtra los DataFrames originales utilizando los correos electrónicos repetidos
    df1_interseccion = df1[df1['Email'].isin(emails_repetidos)]
    df2_interseccion = df2[df2['Email'].isin(emails_repetidos)]
    
    # Crea un nuevo DataFrame con los valores que se repiten en ambos DataFrames
    dataframe_interseccion = pd.concat([df1_interseccion, df2_interseccion], ignore_index=True)
    
    return dataframe_interseccion

In [59]:
info_total = generar_dataframe_interseccion(students_2012_info,students_and_tesis)

['mario.giler@epn.edu.ec', 'kelly.alvarado@epn.edu.ec', 'nelly.herrera@epn.edu.ec', 'jorge.laverde@epn.edu.ec', 'francisco.jacome@epn.edu.ec', 'andres.cruz@epn.edu.ec', 'katherine.vivanco@epn.edu.ec', 'maria.lara@epn.edu.ec', 'katheryn.nicolalde@epn.edu.ec', 'cristian.cabrera@epn.edu.ec', 'bryan.lozano@epn.edu.ec', 'shirley.heredia@epn.edu.ec', 'sharon.larco@epn.edu.ec', 'vicente.capito@epn.edu.ec', 'veronica.torres@epn.edu.ec', 'dennis.arequipa@epn.edu.ec', 'karen.paguanquiza@epn.edu.ec', 'david.jacome@epn.edu.ec', 'alexis.guzman@epn.edu.ec', 'michelle.cruz@epn.edu.ec', 'victor.sandoval@epn.edu.ec', 'santiago.lema@epn.edu.ec', 'paul.bermudez@epn.edu.ec', 'jorge.campos@epn.edu.ec', 'jonathan.salazar@epn.edu.ec', 'christian.quezada@epn.edu.ec', 'maria.sangucho@epn.edu.ec', 'jonathan.franco@epn.edu.ec', 'nathaly.cumbicos@epn.edu.ec', 'nayla.velasquez@epn.edu.ec', 'yadira.yapo@epn.edu.ec', 'francisco.hidalgo@epn.edu.ec', 'cristina.pauta@epn.edu.ec', 'jimmy.pardo@epn.edu.ec', 'marcelo.maya

In [60]:
print(info_total)

            APELLIDOS           NOMBRES  \
0       ABARCA ROBLES       JUAN DANIEL   
1     AGUILAR ALVAREZ    SANDRA NATHALY   
2    AGUILAR ENRÍQUEZ  MIGUEL ALEJANDRO   
3       AGUIRRE CUEVA  KLEBER ALEXANDER   
4         AGUIRRE IZA   STALIN PATRICIO   
..                ...               ...   
629               NaN               NaN   
630               NaN               NaN   
631               NaN               NaN   
632               NaN               NaN   
633               NaN               NaN   

                                               CARRERA  \
0                            INGENIERIA AGROINDUSTRIAL   
1        INGENIERIA ELECTRONICA Y REDES DE INFORMACION   
2                                           MATEMATICA   
3        INGENIERIA ELECTRONICA Y REDES DE INFORMACION   
4    INGENIERIA EN SISTEMAS INFORMATICOS Y DE COMPU...   
..                                                 ...   
629                                                NaN   
630                

## Load

In [22]:
if __name__ == '__main__':
    getInfoFromPdf()
    dataset = getDatasetFromCSVFile(FILE_PATH_STUDENTS_CSV)
    dataset = addEmail(dataset)
    print(dataset)

            APELLIDOS               NOMBRES                    CARRERA  \
0       ABARCA ROBLES           JUAN DANIEL  INGENIERIA AGROINDUSTRIAL   
1     ACOSTA CEVALLOS        STALIN ROBERTO                 MATEMATICA   
2      ACOSTA ORDOÑEZ  CHRISTOPHER LEONARDO        INGENIERIA MECANICA   
3       ACOSTA ROSERO           LUIS FELIPE        INGENIERIA MECANICA   
4       ACUÑA MONTERO         MARLON ANDRES                     FISICA   
..                ...                   ...                        ...   
768    ZAPATA HERRERA      CHRISTIAN JAVIER        INGENIERIA MECANICA   
769    ZAPATA HIDALGO        JOSSELYN PAOLA  INGENIERIA AGROINDUSTRIAL   
770      ZAPATA TERAN     STEPHANY CAROLINA     INGENIERIA EMPRESARIAL   
771  ZURITA FERNANDEZ         DIEGO ESTEBAN    INGENIERIA EN PETROLEOS   
772       ZURITA YUGS          KEVIN JAVIER        INGENIERIA MECANICA   

                                 email  
0           juan.abarca@epn.edu.ec.com  
1         stalin.acosta@epn.e